# Assignment 4: "Search your transcripts. You will know it to be true."

© Cristian Danescu-Niculescu-Mizil 2023

<p style="text-align:center">
    <img src="https://imgs.xkcd.com/comics/mainly_known_for.png" alt="xkcd Mainly Known For: Oh sure, I know Keira Knightly, from the first movie in that series by The Land Before Time producer. You know, the franchise with the guy from Jurassic Park and Ghostwriter, and script work by Billie Lourd's mom?" width="650"/><br/>(source: <a href="https://xkcd.com/2621">https://xkcd.com/2621</a>)
</p>

## CS/INFO 4300 Language and Information<a class="anchor" id="guidelines"></a>


<font color='red'>

**DUE**: February 22, 2023 (Wednesday), 11:59pm
    
</font>

<font color='purple'> 
    
**EXTRA CREDIT QUESTION** due: February 24th, 2023 (Friday), 11:59pm (separate CMS submission, same notebook)

</font>

This is an **individual** assignment. 

If you use any outside sources (e.g. research papers, StackOverflow) please list your sources.

Need to find out the full name of "steve pixar guy"? Or perhaps you want to know the identity of "song guy from labyrinth"? Sounds like what you need is a system that can efficiently search through large collections of documents!

In class, we have seen a number of techniques useful for searching through collections of documents, including edit distance and cosine similarity. Our overall goal in this assignment is to apply these techniques to build a system that efficiently searches for documents similar to a query in large data sets. We will explore the tradeoffs of information retrieval systems by finding newspaper quotes from "Keeping Up With The Kardashians".

**Guidelines**

All cells that contain the blocks that read `# YOUR CODE HERE` are editable and are to be completed to ensure you pass the test-cases. Make sure to write your code where indicated.

All cells that read `YOUR ANSWER HERE` are free-response cells that are editable and are to be completed.

You may use any number of notebook cells to explore the data and test out your functions, although you will only be graded on the solution itself.

You are unable to modify the read-only cells and should never delete any of the given cells.

You should also use Markdown cells to explain your code and discuss your results when necessary.
Instructions can be found [here](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html).

All floating point values should be printed with **2 decimal places** precision. You can do so using the built-in round function.

Unless otherwise stated, no cell in this assignment should take longer than **1 second** to run. If a cell takes longer than 1 second to run, it will be marked as incorrect.

**Grading**

For code-completion questions you will be graded on passing the public test cases we have included, as well as any hidden test cases that we have supplemented to ensure that your logic is correct.

For free-response questions you will be manually graded on the quality of your answer.

**Learning Objectives**
- Examine how edit distance can be applied as a search metric
- Develop an understanding of the inverted index and its applications
- Explore use cases of boolean search
- Examine how the inverted index can be used to efficiently compute IDF values
- Introduce cosine similarity as an efficient search model

- - -
### Contents  <a class="anchor" id="contents"></a>

* [Instructions and guidelines](#guidelines)
* [Contents](#contents)
* [Imports and Motivations](#imports)


* [[**Minimum Edit Distance**]](#part1)
  * [Imports and data](#0)
  * [[1](#q1a)] Levenshtein Edit Distance
    * [[1a](#q1a)] Levenshtein Edit Distance
    * [[1b](#q1b)] Query Discussion and Analysis
  * [[2](#q2a)] Change Levenshtein Edit Distance Costs
    * [[2a](#q2a)] Changing the costs
    * [[2b](#q2b)] Query Discussion and Analysis
    
* [[**Cosine Similarity**]](#part2)
  * [[3](#q3)] Inverted Index
    * [[3](#q3)] Write a function to construct the inverted index
  * [[4](#q4a)] Inverted Index and Boolean Search
    * [[4a](#q4a)] Using the inverted index for boolean search
    * [[4b](#q4b)] Using the inverted index for boolean search
  * [[5](#q5)] Inverted Index and IDF
    * [[5](#q5)] Compute IDF using the inverted index
  * [[6](#q6)] Inverted Index and Norm of Document
    * [[6](#q6)] Compute the norm of each document using the inverted index
  * [[7](#q7)] Inverted Index and Dot Product
    * [[7](#q7)] Efficiently compute the dot product using the inverted index
  * [[8](#q8)] Similar Message Finder
    * [[8a](#q8)] Find the most similar messages to the quotes
    * [[8b](#q8b)] Find the most similar messages to the quotes analysis
    
* [[**Extra Credit (optional)**]](#ec)  
  * [[9](#q9)] Finding your own similarity metric


(For those curious, the rendering of backlink to "contents" from each section, i.e., the right corner [[contents](#contents)] link, is only accessible via .ipynb notebook, it causes unexpected behavior when viewed in .html.)

### Imports <a class="anchor" id="imports"></a>

In [31]:
from collections import defaultdict
from collections import Counter
import json
import math
import string
import time
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from IPython.core.display import HTML

In [32]:
with open("kardashian-transcripts.json", "r") as f:
    transcripts = json.load(f)
print(len(transcripts[0]))

851


# Find Most Similar Messages

## Motivation

The tabloids have been going crazy over our stars! The press took some  quotes from the show, including:
       
 - *"It's like a bunch of people running around talking about nothing."*
 - *"Never say to a famous person that this possible endorsment would bring 'er to the spot light."*
 - *"Your yapping is making my head ache!"*
 - *"I'm going to Maryland, did I tell you?"*
 
We need to find out who said each of these, and in which episode. But since we're information scientists, that's not enough. We want to build an efficient search engine for retrieving where such quotes come from in the future.

What makes this difficult is that journalists often modify the quotes, so exact matching will not always work.

In [33]:
treebank_tokenizer = TreebankWordTokenizer()
flat_msgs = [m for transcript in transcripts for m in transcript]
queries = [u"It's like a bunch of people running around talking about nothing.",
           u"Never say to a famous person that this possible endorsment would bring 'er to the spot light.",
           u"Your yapping is making my head ache!",
           u"I'm going to Maryland, did I tell you?"]

- - -
<p style='text-align:right; font-size:11px'>[[contents](#contents)]</p>

## Minimum Edit Distance<a class="anchor" id="part1"></a>

### Q1a: Levenshtein Edit Distance (Code Completion)<a class="anchor" id="q1a"></a>

From class and previous assignments, you have learned several methods of comparing similarities between text. Here, we will now first try using the edit distance metric (Levenshtein) to see how it compares the query text with our dataset. For each query, we will need to loop over each message, and compute the edit distance each time, there are no obvious shortcuts. (The code will take some time to run!)

We've provided you with code to compute the edit matrix that we discussed in class. You will need to complete the code stub below. (You can refer to the worksheet we used in class if you need a refresher on how this works.)

Note that instead of directly coding the insertion, deletion, and substitution costs into the edit matrix code, we've designed the code to take three *functions* as parameters to compute each cost. You'll see why we did it this way once you get to the next question! As in previous assignments, make sure to use these parameters; you will lose points if you hard-code the costs or reference global variables!

Note: It is ok for there to be duplicates in your results.

In [36]:
def insertion_cost(message, j):
    return 1

def deletion_cost(query, i):
    return 1

def substitution_cost(query, message, i, j):
    if query[i-1] == message[j-1]:
        return 0
    else:
        return 1

def edit_matrix(query, message, ins_cost_func, del_cost_func, sub_cost_func):
    """ Calculates the edit matrix
    
    Arguments
    =========
    
    query: query string,
        
    message: message string,
    
    ins_cost_func: function that returns the cost of inserting a letter,
    
    del_cost_func: function that returns the cost of deleting a letter,
    
    sub_cost_func: function that returns the cost of substituting a letter,
    
    Returns:
        edit matrix {(i,j): int}
    """
    
    m = len(query) + 1
    n = len(message) + 1

    chart = {(0, 0): 0}
    for i in range(1, m): 
        chart[i,0] = chart[i-1, 0] + del_cost_func(query, i) 
    for j in range(1, n): 
        chart[0,j] = chart[0, j-1] + ins_cost_func(message, j)
    for i in range(1, m):
        for j in range(1, n):
            chart[i, j] = min(
                chart[i-1, j] + del_cost_func(query, i),
                chart[i, j-1] + ins_cost_func(message, j),
                chart[i-1, j-1] + sub_cost_func(query, message, i, j)
            )
    return chart

def edit_distance(query, message, ins_cost_func, del_cost_func, sub_cost_func):
    """ Finds the edit distance between a query and a message using the edit matrix
    
    Arguments
    =========
    
    query: query string,
        
    message: message string,
    
    ins_cost_func: function that returns the cost of inserting a letter,
    
    del_cost_func: function that returns the cost of deleting a letter,
    
    sub_cost_func: function that returns the cost of substituting a letter,
    
    Returns:
        edit cost (int)
    """
        
    query = query.lower()
    message = message.lower()
    
    # YOUR CODE HERE

    d = edit_matrix(query, message,ins_cost_func, del_cost_func, sub_cost_func)
    
    return d[len(query), len(message)]

def edit_distance_search(query, msgs, ins_cost_func, del_cost_func, sub_cost_func):
    """ Edit distance search
    
    Arguments
    =========
    
    query: string,
        The query we are looking for.
        
    msgs: list of dicts,
        Each message in this list has a 'text' field with
        the raw document.
        
    ins_cost_func: function that returns the cost of inserting a letter,
    
    del_cost_func: function that returns the cost of deleting a letter,
    
    sub_cost_func: function that returns the cost of substituting a letter,
    
    Returns
    =======
    
    result: list of (score, message) tuples.
        The result list is sorted by score such that the closest match
        is the top result in the list.
    
    """
    # YOUR CODE HERE
    returnList = []
    
    for msg in msgs:
        score = edit_distance(query, msg['text'],ins_cost_func, del_cost_func, sub_cost_func)
        returnList.append( (score, msg) )
        
    returnList.sort(key=lambda x:x[0])
    
    return returnList

**NOTE**: Autograder tests for this function may take a few minutes to execute.

In [37]:
# This is an autograder test. Here we can test the function you just wrote above.
score, _ = edit_distance_search(queries[1], flat_msgs, insertion_cost, deletion_cost, substitution_cost)[0]
assert score == 42


When the edit_distance_search function is completed you can run the lines below to print out the best matches for each query string. **NOTE**: This cell will take a few minutes to execute.

In [38]:
# note: this will take about 1-2 minutes to execute
top_10 = []
for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    for score, msg in edit_distance_search(query, flat_msgs, insertion_cost, deletion_cost, substitution_cost)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            msg['speaker'],
            msg['text'],
            msg['episode_title']))
        top_10.append(msg)
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[0.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[33.00] KRIS: It's not a bunch of teenagers running around.
	(Keeping Up With the Kardashians - Kris ``The Cougar'' Jenner)
[35.00] KHLOE: It's like, what are you talking about?
	(The Wedding: Keeping Up With the Kardashians)
[37.00] KIM: It's like it has separation anxiety or something.
	(Keeping Up With the Kardashians - Shape Up or Ship Out)
[37.00] KHLOE: It's like I want to learn how to do that.
	(Keeping Up With the Kardashians - Distance Makes the Heart Grow Fonder)
[37.00] KOURTNEY: It's like an explosion in your pantyhose.
	(Keeping Up With the Kardashians - I'd Rather Go Naked... Or Shopping)
[38.00] ROB: I have a bunch of connections in the indus

### Q1b: Query Discussion and Analysis (Free Response)<a class="anchor" id="q1b"></a>

What do you notice about the search above?
1. Discuss why it worked, or why it might not have worked, for each query. 

2. Do you notice anything different about the costs to those discussed in lecture? 

3. Please fill out the table below with the costs from running the Levenshtein algorithm discussed in lecture and the cost from using the edit_distance function we provided above. Copy-and-paste this markdown table into the cell below for your answers:
```
| Operation    | Cost (Lecture)|  Cost (edit_distance)  | Example
| :----------: |:------------- | :-------------------------- | --------
| Addition     | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "aab"
| Deletion     | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "a"
| None         | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "aa"
| Substitution | YOUR ANSWER   | YOUR ANSWER                 | "aa" -> "ab"
| Multiple     | YOUR ANSWER   | YOUR ANSWER                 | "trawl" --> "claw"
```

YOUR ANSWER HERE

The first query worked well as it found the exact match. As it found "It's not a bunch of teenagers running around." we could see that it works well if there are small changes from the target. 

For the second query, it worked because it found the extremely similar result. The query seems to be the rephrase of what Simon said. Because the query's content and words were quite unique, we could see that it did not find similar matches for others in the list. 

The third query did not work despite it found 2 resemble ones. It is because the query is too short, it found results that have match in non-important words like "You."

The last query also did not work well because it omitted the "maryland," which seems to be the most important word. Same as the third query, it did not work well because the query is very short. 

I realized that the cost for substitution is different to those we used in the lecture. Whereas we used cost of 2 for substitution in the lecture, the cost is 1 this time. Because of change in cost of substitution, the cost of multiple also changed. As the substitution cost decreased, cost for multiple also decreased.



```
| Operation    | Cost (Lecture)|  Cost (edit_distance)  | Example
| :----------: |:------------- | :-------------------------- | --------
| Addition     | 1             | 1                           | "aa" -> "aab"
| Deletion     | 1             | 1                           | "aa" -> "a"
| None         | 0             | 0                           | "aa" -> "aa"
| Substitution | 2             | 1                           | "aa" -> "ab"
| Multiple     | 5             | 3                           | "trawl" --> "claw"
```

### Q2a: Changing the costs (Code Completion)<a class="anchor" id="q2a"></a>

The above implementation has been using standard costs as we did in class. Sometimes we might want to alter the edit-distance penalty depending on the type of change we are making. In practice, the use of a simple uniform cost for all changes is suboptimal, since certain changes are more likely to happen than others. For instance, it is easier to mix up characters that are adjacent to each other on the keyboard (typos), and we'd like you to **reduce the cost of such substitutions of adjacent keyboard letters to 1.5 instead of 2**. You should use the provided list of adjacent keyboard letters, found below.

In [39]:
# we provide you with a list of adjacent characters
adj_chars = [('a', 'q'), ('a', 's'), ('a', 'z'), ('b', 'g'), ('b', 'm'), ('b', 'n'), ('b', 'v'), ('c', 'd'),
             ('c', 'v'), ('c', 'x'), ('d', 'c'), ('d', 'e'), ('d', 'f'), ('d', 's'), ('e', 'd'), ('e', 'r'),
             ('e', 'w'), ('f', 'd'), ('f', 'g'), ('f', 'r'), ('f', 'v'), ('g', 'b'), ('g', 'f'), ('g', 'h'),
             ('g', 't'), ('h', 'g'), ('h', 'j'), ('h', 'm'), ('h', 'n'), ('h', 'y'), ('i', 'k'), ('i', 'o'),
             ('i', 'u'), ('j', 'h'), ('j', 'k'), ('j', 'u'), ('k', 'i'), ('k', 'j'), ('k', 'l'), ('l', 'k'),
             ('l', 'o'), ('m', 'b'), ('m', 'h'), ('n', 'b'), ('n', 'h'), ('o', 'i'), ('o', 'l'), ('o', 'p'),
             ('p', 'o'), ('q', 'a'), ('q', 'w'), ('r', 'e'), ('r', 'f'), ('r', 't'), ('s', 'a'), ('s', 'd'),
             ('s', 'w'), ('s', 'x'), ('t', 'g'), ('t', 'r'), ('t', 'y'), ('u', 'i'), ('u', 'j'), ('u', 'y'), 
             ('v', 'b'), ('v', 'c'), ('v', 'f'), ('w', 'e'), ('w', 'q'), ('w', 's'), ('x', 'c'), ('x', 's'), 
             ('x', 'z'), ('y', 'h'), ('y', 't'), ('y', 'u'), ('z', 'a'), ('z', 'x')]

In [40]:
def substitution_cost_adj(query, message, i, j):
    """
    Custom substitution cost:
    The cost is 1.5 when substituting a pair of characters that can be found in adj_chars
    Otherwise, the cost is 2. (Not 1 as it was before!)
    """
    # YOUR CODE HERE

    if query[i-1] == message[j-1]:
        return 0 
    elif (query[i-1],message[j-1]) in adj_chars:
        return 1.5 
    else:
        return 2

In [41]:
assert edit_distance("Levenshtein","Levenstein", insertion_cost, deletion_cost, substitution_cost_adj) == 1
assert edit_distance("Levenshtein","Lebenshtein", insertion_cost, deletion_cost, substitution_cost_adj) == 1.5

Run the following code below to reorder the top ten closest matches for each query using our new custom edit-distance discounts. Once again, this may take a few minutes to run.

Note: It is ok to obtain duplicate results.

In [42]:
for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    # we compare the query with top_10 to save time because
    # we already know that top_10 will be close to the query
    # so there is no need to check all flat_msgs
    for score, msg in edit_distance_search(query, top_10, insertion_cost, deletion_cost, substitution_cost_adj)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            msg['speaker'],
            msg['text'],
            msg['episode_title']))
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[0.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[39.50] KRIS: It's not a bunch of teenagers running around.
	(Keeping Up With the Kardashians - Kris ``The Cougar'' Jenner)
[42.00] KHLOE: It's like, what are you talking about?
	(The Wedding: Keeping Up With the Kardashians)
[45.00] KOURTNEY: It's like an explosion in your pantyhose.
	(Keeping Up With the Kardashians - I'd Rather Go Naked... Or Shopping)
[47.00] KHLOE: It's like I want to learn how to do that.
	(Keeping Up With the Kardashians - Distance Makes the Heart Grow Fonder)
[48.00] BRUCE: That's why you're running around wearing black.
	(Keeping Up With the Kardashians - I'd Rather Go Naked... Or Shopping)
[48.00] BRUCE: That's why you're running 

### Q2b: Query Discussion and Analysis (Free Response)<a class="anchor" id="q2b"></a>

4. How do the results after cost changes different compare to that of default `edit_distance` metric?

YOUR ANSWER HERE

For the first query, there are some results that have been influenced by the factor of adjacnet keyboard letters. Whereas "[37.00] KIM: It's like it has separation anxiety or something." was ranked 4th place in the default metric, we can see that its rank was pushed back to 8th place. Also, there was a tie between KHOLE and KOURTNEY's script with the cost of [37.00] in the default metric. After the manipulation, we can see that KHLOE's now cost more, probably assuming that words were not adjacent compared to KOURTNEY's. 

For the second query, it is interesting that [59.00] KRIS does not even appear from the result after the cost change. I could infer that KRIS's words were not adjacent as the others and affected a lot. 

For the third query, we can see that the ties between BRUCE and KHOLE has been ranked. Bruce's line were highly affected by the adjacency so that it does not appear after manipulation. For the ties with BRUCE and KHLOE with the cost of [22.00], we can see they are now also ranked. Also, a new script line of [34.50] ROB: appeared on a list as it probably had lesser cost with adjacent words. 

Lastly, we initially had 5 ties with cost of [19.00] in a default metric. Now, they all have different values exept the duplication, with the cost of [26.5], [27.00] and [27.5] after the modification by the adjacency.

- - -
<p style='text-align:right; font-size:11px'>[[contents](#contents)]</p>

## Cosine Similarity<a class="anchor" id="part2"></a>

### A high-level overview

Our overall goal of this assignment is to build a system where we can compute the text similarity between queries and our datasets quickly. From the previous part, we tried using variations of minimum edit distance as the metric. Now, we will try another method -- comparing two text through cosine similarity score. To accomplish queries and compute cosine similarities, as we learned in class, we will need to represent documents as ***vectors***! A common method of representing documents as vectors is by using "term frequency-inverse document frequency" (TF-IDF) scores. (Check the lecture modules on Canvas if you need a refresher). The notation here is consistent with the in-class hand out, so if you haven't read over it -- you should!

Consider the TF-IDF representation of a document and a query: $\vec{d_j}$ and $\vec{q}$, respectively. Elements of these vectors are very often zero because the term frequency of most words in most documents is zero. Stated differently, most words don't appear in most documents! Consider a query that has 5 words in it and a vocabulary that has 20K words in it -- only .025% of the elements of the vector representation of the query are nonzero! When a vector (or a matrix) has very few nonzero entries, it is called "sparse." We can take advantage of the sparsity of tf-idf document representations to compute cosine similarity quickly. We will first build some data stuctures that allow for faster querying of statistics, and then we will build a function that quickly computes cosine similarity between queries and documents.

### A starting point
We will use an **inverted index** for efficiency. This is a sparse term-centered representation that allows us to quickly find all documents that contain a given term.

### Q3 Write a function to construct the inverted index (Code Completion)<a class="anchor" id="q3"></a>

As in class, the inverted index is a key-value structure where the keys are terms and the values are lists of *postings*. In this case, we record the documents a term occurs in as well as the **count** of that term in that document.

In [43]:
def build_inverted_index(msgs):
    """ Builds an inverted index from the messages.
    
    Arguments
    =========
    
    msgs: list of dicts.
        Each message in this list already has a 'toks'
        field that contains the tokenized message.
    
    Returns
    =======
    
    inverted_index: dict
        For each term, the index contains 
        a sorted list of tuples (doc_id, count_of_term_in_doc)
        such that tuples with smaller doc_ids appear first:
        inverted_index[term] = [(d1, tf1), (d2, tf2), ...]
        
    Example
    =======
    
    >> test_idx = build_inverted_index([
    ...    {'toks': ['to', 'be', 'or', 'not', 'to', 'be']},
    ...    {'toks': ['do', 'be', 'do', 'be', 'do']}])
    
    >> test_idx['be']
    [(0, 2), (1, 2)]
    
    >> test_idx['not']
    [(0, 1)]
    
    """
    # YOUR CODE HERE

    inverted_index = {} # key: toks, value: list of tokened msgs 
        
    for doc_id, msg in enumerate(msgs):
        
        listofTerms = msg['toks'] #list of tokened msgs 
        countTerms = Counter(listofTerms)
                
        for k,v in countTerms.items():
            if k in inverted_index.keys():
                inverted_index[k].append((doc_id,v))
            else:
                inverted_index[k] = [(doc_id, v)]
                
    return inverted_index

In [44]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
inv_idx = build_inverted_index(flat_msgs)
execution_time = time.time() - start_time

assert len(inv_idx) <= 10000 
assert [i[0] for i in inv_idx['bruce']] == sorted([i[0] for i in inv_idx['bruce']])
assert len(inv_idx['bruce']) < len(inv_idx['kim'])
assert len(inv_idx['bruce']) >= 400 and len(inv_idx['bruce']) <= 435
assert len(inv_idx['baby']) >= 250 and len(inv_idx['baby']) <= 300
assert execution_time <= 1.0


### Q4a Using the inverted index for boolean search (Code Completion)<a class="anchor" id="q4a"></a>

In this section we will use the inverted index you constructed to perform an efficient boolean search. The boolean model was one of the early information retrieval models, and continues to be used in applications today.

A boolean search works by searching for documents which match the boolean expression of the query. Three main operators in a boolean search are `AND` `OR` and `NOT`. For example, the query `"Ned" AND "Rob"` would return any document which contains both the words "Ned" and "Rob".

Here, we will treat a query as a simple two-word search with exclusion---that is, we will implement a search with `NOT` instead of `AND`. For example, the query words "kardashian", "kim" would be equivalent to the boolean expression `"kardashian" NOT "kim"`, and will return any document which contains the word "kardashian" while not containing the word "kim".

#### In class we implemented the Merge Postings Algorithm, review the code [here](https://colab.research.google.com/drive/1Egg26OyynPuWizjEhSzMJ4rad9TwFQ1s?usp=sharing).
Reminder: the algorithm takes two postings lists (A and B), then proceeds as follows:

------------------------------------------
Initialize empty list (called merged list M)

Start: Pointer at the first element of both A and B

Do: Does it point to the same document ID in each list?

    Yes: Append document ID to M, advance pointer in both A and B
    No: Advance (to the right) the pointer with the smaller ID
    
End: When we attempt to advance a pointer already at the end of its list

------------------------------------------

The above Merge Postings Algorithm can be thought of a boolean search with the `AND` operator. Write a function `boolean_search` that implements a similar algorithm with the `NOT` operator using the inverted index.

**Note:** Make sure you convert the `query_word` and `not_word` to lowercase. 


In [45]:
def boolean_search(query_word,not_word, inverted_index):
    """ Search the collection of documents for the given query_word 
        provided that the documents do not include the not_word
    
    Arguments
    =========
    
    query_word: string,
        The word we are searching for in our documents.
    
    not_word: string,
        The word excluded from our documents.
    
    index: an inverted index as above
    
    
    Returns
    =======
    
    results: list of ints
        Sorted List of results (in increasing order) such that every element is a `doc_id`
        that points to a document that satisfies the boolean
        expression of the query.
        
    """
    # YOUR CODE HERE
    
    returnList = [] #list of ints 

    query = query_word.lower()
    q = sorted(inverted_index[query])

    excluded = not_word.lower()
    e = sorted(inverted_index[excluded])
    
    i=0 
    j=0
    
    while i<len(q) and j<len(e):
        
        #don't appened, advance both pointer 
        if q[i][0]==e[j][0]: 
            i+=1
            j+=1 
        
        #advance smaller ID
        elif q[i][0]<e[j][0]: 
            returnList.append(q[i][0])
            i+=1 
        
        else:
            j+=1 
        
    if i<len(q):
        for r in range(i,len(q)):
            returnList.append(q[r][0])
    
    return returnList

In [46]:
result0_start_time = time.time()
result0 = boolean_search('ice','cream', inv_idx)
result0_execution_time = time.time() - result0_start_time
result3 = boolean_search('puppy','dog', inv_idx)
result1= boolean_search('Kardashian','Kim',inv_idx)
result4= boolean_search('cake','cake',inv_idx)
assert result0_execution_time < 1.0
assert type(result1) == list
assert len(result3) == 7
assert len(result4)==0


### Q4b Using the inverted index for boolean search (Free Response)<a class="anchor" id="q4b"></a>

Earlier in the assignment, we already explored search techniques (edit distance) which are able to find a wider variety of relevant results. Why might you want to use a boolean search with an inverted index instead? Justify your answer by explaining the difference between edit distance and boolean search with inverted index, *and* including at least one example where boolean search with inverted index would be a better choice than an edit distance search. **You will lose points if you do not provide an example!**

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

Using inverted index is more efficient because it allows to quickly find all documents that contain a given term as it is term-based. As described, most of words do not appear in most of the documents. If we use the edit distance, we would have to go through all document and calculate the edit distance, which is time consuming. In contrast, when we use inverted index, we only look at the documents that contain the word, so it is more efficient. For instance, let's say there is a "hello" query, which appears only in 5 documents out of 1,000 documents in total.  If we use edit distance, we would go over all 1,000 documents and find out there is only 5 documents that contains "hello." This will take huge amount of time. However, when we use the boolean search with inverted index, we would start off with the 5 documents that contains "hello". Likewise, using boolean search with inverted index would be the better choice. 

<div style="border-top: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold; text-align: center;"></div>

### Q5 Compute IDF *using* the inverted index (Code Completion)<a class="anchor" id="q5"></a>

Write a function `compute_idf` that uses the inverted index to efficiently compute IDF values.

Words that occur in a very small number of documents are not useful in many cases, so we ignore them. Use a parameter `min_df`
to ignore all terms that occur in strictly fewer than `min_df=10` documents.

Similarly, words that occur in a large *fraction* of the documents don't bring any more information for some tasks. Use a parameter `max_df_ratio` to trim out such words. For example, `max_df_ratio=0.95` means ignore all words that occur in more than 95% of the documents.

As a reminder, we define the IDF statistic as...
$$ IDF(t) = \log \left(\frac{N}{1 + DF(t)} \right) $$

where $N$ is the total number of docs and $DF(t)$ is the number of docs containing $t$. Keep in mind, there are other definitions of IDF out there, so if you go looking for resources on the internet, you might find differing (but also valid) accounts. In practice the base of the log doesn't really matter, however you should use base 2 here.

In [47]:
def compute_idf(inv_idx, n_docs, min_df=10, max_df_ratio=0.95):
    """ Compute term IDF values from the inverted index.
    Words that are too frequent or too infrequent get pruned.
    
    Hint: Make sure to use log base 2.
    
    Arguments
    =========
    
    inv_idx: an inverted index as above
    
    n_docs: int,
        The number of documents.
        
    min_df: int,
        Minimum number of documents a term must occur in.
        Less frequent words get ignored. 
        Documents that appear min_df number of times should be included.
    
    max_df_ratio: float,
        Maximum ratio of documents a term can occur in.
        More frequent words get ignored.
    
    Returns
    =======
    
    idf: dict
        For each term, the dict contains the idf value.
        
    """
    
    # YOUR CODE HERE
    
    idf = dict()
    
    #iterate through all query terms
    for i in inv_idx:
        
        count = len(inv_idx[i])
        
        if (count < min_df or  count/n_docs > max_df_ratio):
            continue
            
        else:
            result = math.log2(n_docs/(1+len(inv_idx[i])))
            idf[i] = round(result,2)
    
    return idf

In [48]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
idf_dict = compute_idf(inv_idx, len(flat_msgs))
execution_time = time.time() - start_time

assert len(idf_dict) < len(inv_idx)
assert 'blah' not in idf_dict
assert 'blah' in inv_idx 
assert '.' in idf_dict
assert '3' not in idf_dict
assert idf_dict['bruce'] >= 6.0 and idf_dict['bruce'] <= 7.0
assert idf_dict['baby'] >= 6.0 and idf_dict['baby'] <= 8.0
assert execution_time <= 1.0


### Q6 Compute the norm of each document using the inverted index (Code Completion)<a class="anchor" id="q6"></a>

Recalling our tf-idf vector representation of documents, we can compute the "norm" as the norm (length) of the vector representation of that document. More specifically, the norm of a document $j$, denoted as $\left|\left| d_j \right|\right|$, is given as follows...

$$ \left|\left| d_j \right|\right| = \sqrt{\sum_{\text{word } i} (tf_{ij} \cdot idf_i)^2} $$

Once again, this can be quite efficiently implemented using the inverted index. Below, write a function that does this.

In [49]:
def compute_doc_norms(index, idf, n_docs):
    """ Precompute the euclidean norm of each document.
    
    Arguments
    =========
    
    index: the inverted index as above
    
    idf: dict,
        Precomputed idf values for the terms.
    
    n_docs: int,
        The total number of documents.
    
    Returns
    =======
    
    norms: np.array, size: n_docs
        norms[i] = the norm of document i.
    """
    
    # YOUR CODE HERE
     
    norms = np.zeros(n_docs)
            
    for k,v in idf.items():
        for docId, count in index[k]:
            norms[docId] += (count*idf[k])**2
        
    return np.sqrt(norms)

In [50]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
doc_norms = compute_doc_norms(inv_idx, idf_dict, len(flat_msgs))
execution_time = time.time() - start_time

assert len(flat_msgs) == len(doc_norms)
assert doc_norms[3722] == 0
assert max(doc_norms) < 80
assert doc_norms[1] >= 15.5 and doc_norms[1] <= 17.5
assert doc_norms[5] >= 6.5 and doc_norms[5] <= 8.5
assert execution_time <= 1.0


### Q7 Efficiently compute the dot product *using* the inverted index (Code Completion)<a class="anchor" id="q7"></a>

We are getting closer to being able to use our inverted index to do a fast implementation of cosine similarity. There is one more ingredient that will come in handy when it comes to optimization. Recall the definition of cosine similarity (where ${q}$ is a query tf-idf vector and ${d_j}$ is the tf-idf vector of the ${j}$th document):

$$ cossim(\vec{q}, \vec{d_j}) = \frac{\vec{q} \cdot \vec{d_j}}{\|\vec{q}\| \cdot \|\vec{d_j}\|}$$

Specifically, let's focus on the numerator, which computes a dot product. If we recall the definition of the dot product, this can be written as:

$$ \vec{q} \cdot \vec{d_j} = \sum_{i} {q_i} * {d_i}_j $$

Here ${q_i}$ and ${d_i}_j$ are the $i$-th dimension of the vectors $q$ and ${d_j}$ respectively.
Because many ${q_i}$ and ${d_i}_j$ are zero, it is actually a bit wasteful to actually create the vectors $q$ and $d_j$ as numpy arrays; this is the method that you saw in class.

A faster approach to computing the numerator term (dot product) for cosine similarity involves quickly computing the above summation using the inverted index. Recall from class that this is achieved via a *term-at-a-time* approach, iterating over ${q}_j$ that are nonzero (i.e. ${q}_j$ such that the word $j$ appears in the query) and building up *score accumulators* for each document as you iterate.

In this question, you will write a function that implements this term-at-a-time score accumulation.

In [51]:
def accumulate_dot_scores(query_word_counts, index, idf):
    """ Perform a term-at-a-time iteration to efficiently compute the numerator term of cosine similarity across multiple documents.
    
    Arguments
    =========
    
    query_word_counts: dict,
        A dictionary containing all words that appear in the query;
        Each word is mapped to a count of how many times it appears in the query.
        In other words, query_word_counts[w] = the term frequency of w in the query.
        You may safely assume all words in the dict have been already lowercased.
    
    index: the inverted index as above,
    
    idf: dict,
        Precomputed idf values for the terms.
    
    Returns
    =======
    
    doc_scores: dict
        Dictionary mapping from doc ID to the final accumulated score for that doc
    """
    
    # YOUR CODE HERE
    
    doc_scores = {} 

    for k,v in query_word_counts.items():
        
        for docId, count in index[k]:
            
            if docId not in doc_scores: 
                doc_scores[docId] = 0
            doc_scores[docId] += idf[k] * count * v * idf[k]
             
    return doc_scores
   

In [52]:
# This is an autograder test. Here we can test the function you just wrote above.
example_query_words = {"like": 2, "mother": 1, "daughter": 1}
start_time = time.time()
scores = accumulate_dot_scores(example_query_words, inv_idx, idf_dict)
execution_time = time.time() - start_time

assert len(scores) > 3000
assert max(scores.values()) < 220
assert scores[13] > 27 and scores[13] < 30
assert scores[324] > 55 and scores[324] < 58
assert execution_time <= 1.0


### Q8a Find the most similar messages to the quotes (Code Completion)<a class="anchor" id="q8"></a>

The goal of this section is to implement `index_search`, a fast implementation of cosine similarity. You will then test your answer by running the search function using the code provided. Briefly discuss why it worked, or why it might not have worked, for each query.

The goal of `index_search` is to compute the cosine similarity between the query and each document in the dataset. Naively, this computation requires you to compute dot products between the query tf-idf vector $q$ and each document's tf-idf vector $d_i$, which sounds very computationally expensive!

Thankfully, you should be able to use the sparsity of the tf-idf representation and the data structures you created to your advantage. We have already laid down all the groundwork we need for this in the previous questions---all that is left to do here is to put it all together!

Once again, recall that our goal is to compute the cosine similarity:

$$ cossim(\vec{q}, \vec{d_j}) = \frac{\vec{q} \cdot \vec{d_j}}{\|\vec{q}\| \cdot \|\vec{d_j}\|}$$

Just to recap, thus far we have created the following:
- An inverted index mapping from word to the documents they appear in
- IDF values for every word that does not appear too rarely or too frequently
- Vector norms for the tf-idf vectors of every document
- A function that uses term-at-a-time iteration to efficiently compute dot products between the query and multiple documents (that is, the numerator of the cosine similarity)

Your task in this function will be to assemble the above components to compute the cosine similarities between the query and documents, and use these to rank the documents.

**Note:** Convert the query to lowercase, and use the `nltk.tokenize.TreebankWordTokenizer` to tokenize the query (provided to you as the `tokenizer` parameter). The transcripts have already been tokenized this way. <br>

**Note 2:** For `index_search`, you need not remove punctuation tokens from the tokenized query before searching.

**Aside:** Precomputation

In many settings, we will need to repeat the same kind of operation many times. Often, part of the input doesn't change.
Queries against the Kardashians transcript are like this: we want to run more queries (in the real world we'd want to run a lot of them every second, even) but the data we are searching doesn't change.

We could write an `index_search` function which taking the `query` and the `msgs` as input, and the function would look like:

    def index_search(query, msgs):
        inv_idx = build_inverted_index(msgs)
        idf = compute_idf(inv_idx, len(msgs))
        doc_norms = compute_doc_norms(inv_idx)
        # do actual search


But notice that the first three lines only depend on the messages. Imagine if we run this a million times with different queries but the same collection of documents: we'd wastefully recompute the index, the IDFs and the norms every time and discard them. It's a better idea, then, to precompute them just once, and pass them as arguments.

In [53]:
inv_idx = build_inverted_index(flat_msgs)

idf = compute_idf(inv_idx, len(flat_msgs),
                  min_df=10,
                  max_df_ratio=0.1)  # documents are very short so we can use a small value here
                                     # examine the actual DF values of common words like "the"
                                     # to set these values

inv_idx = {key: val for key, val in inv_idx.items()
           if key in idf}            # prune the terms left out by idf

doc_norms = compute_doc_norms(inv_idx, idf, len(flat_msgs))

In [60]:
def index_search(query, index, idf, doc_norms, score_func=accumulate_dot_scores, tokenizer=treebank_tokenizer):
    """ Search the collection of documents for the given query
    
    Arguments
    =========
    
    query: string,
        The query we are looking for.
    
    index: an inverted index as above
    
    idf: idf values precomputed as above
    
    doc_norms: document norms as computed above
    
    score_func: function,
        A function that computes the numerator term of cosine similarity (the dot product) for all documents.
        Takes as input a dictionary of query word counts, the inverted index, and precomputed idf values.
        (See Q7)
    
    tokenizer: a TreebankWordTokenizer
    
    Returns
    =======
    
    results, list of tuples (score, doc_id)
        Sorted list of results such that the first element has
        the highest score, and `doc_id` points to the document
        with the highest score.
    
    Note: 
        
    """
    
    # YOUR CODE HERE
    
    results = []
    tokenizedQuery = tokenizer.tokenize(query.lower())
    
    #1. calculate term frequency of query 
    
    tfQuery = {}
    qNorm = 0
    
    for query in tokenizedQuery:
        tfQuery[query] = tfQuery.get(query, 0)+1   
        
        #2. calculate the norm 
        if query in idf.keys():
            qNorm +=np.square(tfQuery[query]*idf[query])
            
    qNorm = np.sqrt(qNorm)
    
    
    #3 calculate
    
    coSim = {}
    
    for query in tokenizedQuery:
        if query in index:
            for doc_id, count in index[query]:
                num = idf[query] * count * tfQuery[query] * idf[query]
                denom = qNorm * doc_norms[doc_id]
                coSim[doc_id] = coSim.get(doc_id, 0) + (num/denom)
        
    for doc_id, score in coSim.items():
        results.append((score, doc_id)) 
        
    results = [x for x in sorted(results, key = lambda x: (-x[0],x[1]))]
        
    return results

In [61]:
# This is an autograder test. Here we can test the function you just wrote above.
start_time = time.time()
results = index_search(queries[1], inv_idx, idf, doc_norms)
execution_time = time.time() - start_time

assert type(results[0]) == tuple
assert max(results)[0] == results[0][0]
assert results[0][0] >= 0.4 and results[0][0] <= 0.48
assert execution_time <= 1.0


for query in queries:
    print("#" * len(query))
    print(query)
    print("#" * len(query))

    for score, msg_id in index_search(query, inv_idx, idf, doc_norms)[:10]:
        print("[{:.2f}] {}: {}\n\t({})".format(
            score,
            flat_msgs[msg_id]['speaker'],
            flat_msgs[msg_id]['text'],
            flat_msgs[msg_id]['episode_title'])) 
    print()

#################################################################
It's like a bunch of people running around talking about nothing.
#################################################################
[1.00] BRUCE: It's like a bunch of people running around talking about nothing.
	(Keeping Up With the Kardashians - Kourt's First Cover)
[0.61] KRIS: It's not a bunch of teenagers running around.
	(Keeping Up With the Kardashians - Kris ``The Cougar'' Jenner)
[0.46] ROB: I have a bunch of connections in the industry.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.46] ROB: I have a bunch of connections in the industry.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.43] KHLOE: She's, like, running.
	(Keeping Up With the Kardashians - Match Made in Hell)
[0.42] ROB: She got me a bunch of interviews.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.42] ROB: She got me a bunch of interviews.
	(Keeping Up With the Kardashians - Must Love Dogs)
[0.41] KHLOE: Not running.
	(Kee

### Q8b Find the most similar messages to the quotes (Free Response)<a class="anchor" id="q8b"></a>

Briefly discuss why cosine similarity worked, or why it might not have worked, **for each query**.

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

- For the first query, the cosine similarity worked because there is the exact match.


- For the second query, it failed to work. For most of the result, it matched the word "light." However, light is not the most important word, it should have search for the "spotlight" added together. I assume that the cossine similarity could not capture the spaces well.


- For the third query, if failed to work. For most of the result, it matched the word "head."I realized from the result of second and third one that cosine similarity focuses on a specific word rather than the overall similarity. 


- It worked for the last query. It found the almost exact match. I could infer that cossine similarity could capture regardless of the word placement. 

<div style="border-top: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold; text-align: center;"></div>

- - -
<p style='text-align:right; font-size:11px'>[[contents](#contents)]</p>

# Extra Credit (Optional)<a class="anchor" id="ec"></a>

### Q9EC (optional)<a class="anchor" id="q9"></a>

### Finding your own similarity metric

We've explored using cosine similarity and edit distance to find similar messages to input queries. However, there's a whole world of ways to measure the similarity between two documents. Go forth, and research!

(Fun fact: Fundamental information retrieval techniques were in fact developed at Cornell, so you would not be the first Cornellian to disrupt the field)

For this question, find a new way of measuring similarity between two documents, and implement a search using your new metric. Your new way of measuring document similarity should be different enough from the two approaches we already implemented. It can be a method you devise or an existing method from somewhere else (make sure to reveal your sources).

**Note:** The amount of EC awarded for this question will be determined based on creativity, originality, implementation, and analysis. *Do not delete the cell below.*

In [1]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 